In [1]:
%%html
<link href="https://v1.fontapi.ir/css/Vazir" rel="stylesheet">
<link rel="stylesheet" href="style.css">

# <div class="farsi center">بسم الله الرحمن الرحیم</div>

## <div class="farsi right">محمد علی صدرایی- 400210993</div>
## <div class="farsi right">محمد مظفری- 400201167</div>
## <div class="farsi right">علیرضا زارع نژاد- 400201101</div>

# <div class="farsi right green"> تعریف توکنایزر‌ها </div>

<div class="farsi right"> برای توکنایز‌ها از توکنایزر‌های کتابخانه Huggingface استفاده شده است.دو توکنایزر در این پروژه استفاده شده است یک توکنایزر در سطح کلمه و یک توکنایزر با استفاده از bpe. </div>
<div class="farsi right"> برای آموزش دادن توکنایزر‌ها از کل ۴ فایل داده شده برای پروژه استفاده شده است. همچنین حداکثر تعداد توکن‌ها برای سطح کلمه ۵۰۰۰ و برای سطح bpe برابر با ۱۰۰۰۰ گذاشته شده است. </div>

In [2]:
import pickle
import glob
import tqdm

In [3]:
TRAIN_TOKENIZERS = False

WORD_TOKENIZER_FILE_NAME = './wtoken.json'
BPE_TOKENIZER_FILE_NAME = './bpetoken.json'

BPE_VOCAB_SIZE = 10000
WORD_LEVEL_VOCAB_SIZE = 5000

UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
SOS_TOKEN = "[SOS]"
EOS_TOKEN = "[EOS]"
ALL_TOKENS = [UNK_TOKEN, SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]

ALL_TRAINING_DATA = [
    './cultural.txt',
    './economics.txt',
    './politics.txt',
    './sports.txt'
]

LM_TRAINING_DATA = ['./train.txt']
LM_TEST_DATA = ['./test.txt']

In [4]:
from collections import defaultdict
test_results = defaultdict(dict)
train_results = defaultdict(dict)

# <div class="green">Tokenization</div>

In [5]:
from tokenizers import Tokenizer
from tokenizers.processors import TemplateProcessing

In [6]:
word_tokenizer = Tokenizer.from_file(WORD_TOKENIZER_FILE_NAME)
bpe_tokenizer = Tokenizer.from_file(BPE_TOKENIZER_FILE_NAME)
def add_post_processor_to(tokenizer: Tokenizer):
    tokenizer.post_processor = TemplateProcessing(
        single=f"{SOS_TOKEN} $0 {EOS_TOKEN}",
        special_tokens=[
            (X, tokenizer.token_to_id(X)) for X in [SOS_TOKEN, EOS_TOKEN]
        ]
    )
    tokenizer.enable_truncation(128)

add_post_processor_to(word_tokenizer)
add_post_processor_to(bpe_tokenizer)

In [7]:
tokenizers = {'word': word_tokenizer, 'bpe': bpe_tokenizer}

# <div class="farsi right green"> خواندن داده‌های تست </div>

In [8]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, corpus_files):
        dataset_lines = []

        for file_name in corpus_files:
            with open(file_name, 'r') as f:
                dataset_lines += f.readlines()
        dataset_lines = [line.strip() for line in dataset_lines]
                
        self.__lines = dataset_lines
        
    def __len__(self):
        return len(self.__lines)
    
    def __getitem__(self, idx):
        return self.__lines[idx]

In [9]:
train_dataset = TextDataset(LM_TRAINING_DATA)
test_dataset = TextDataset(LM_TEST_DATA)

# <div class="farsi right green"> لود کردن مدل‌های n-gram و محاسبه پرپلکسیتی آن </div>

<div class="farsi right">دو مدل n-gram در این پروژه ترین شد. یک مدل با n=3 و laplace=0 و یک مدل با n=3 و laplace=1</div>
<div class="farsi right">در این بخش فایل pkl از پیش ترین شده آن‌ها را لود میکنیم و پرپلکسیتی داده تست را به وسیله آن‌ها محاسبه می‌کنیم.</div>

In [46]:
import argparse
from itertools import product
import math
import nltk
from pathlib import Path
import numpy as np
import itertools
import codecs

class LanguageModel(object):
    def __init__(self, train_data, n, laplace, tokenizer):
        self.set_tokenizer(tokenizer)
        self.n = n
        self.vocab = dict()
        self.laplace = laplace
        self.tokens = self.preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))
    
    def set_tokenizer(self, tokenizer):
        self.sos = str(tokenizer.token_to_id(SOS_TOKEN))
        self.eos = str(tokenizer.token_to_id(EOS_TOKEN))
        self.unk = str(tokenizer.token_to_id(UNK_TOKEN))
        self.tokenizer = tokenizer
    
    def _smooth(self):
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else self.unk for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_tokens_list = self.add_sentence_tokens(test_data, self.n)
        total_len = 0
        total_prob = 0
        
        vocab_size = len(self.vocab)
        
        for test_tokens in tqdm.tqdm(test_tokens_list):
            test_tokens = test_tokens.split()
            test_ngrams = nltk.ngrams(test_tokens, self.n)

            
            known_ngrams  = [self._convert_oov(ngram) for ngram in test_ngrams]
            probabilities = [self.model[ngram]  for ngram in known_ngrams if ngram is not None]
                
            total_len += len(probabilities)
            total_prob += sum(map(math.log, probabilities))
            #for x,y in zip(known_ngrams, probabilities):
             #   print(x,y)
        
        return math.exp((-1/total_len) * total_prob)

    def _best_candidate(self, prev, without=[]):
        
        blacklist  = [self.unk] + without

        if len(prev) < self.n:
            prev = [self.sos]*(self.n-1)

        candidates = list(((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==tuple(prev)))

        probs = [y for x,y in candidates]
        probs = probs/np.sum(probs)
        words = [x for x,y in candidates]

        idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        while words[idx] in blacklist:
            idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        return (words[idx], probs[idx])
         
    def generate_sentence(self, min_len=12, max_len=24):
        sent, prob = ([self.sos] * (max(1, self.n-1)), 1)
        while sent[-1] != self.eos:
            prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
            blacklist = sent + ([self.eos,self.sos] if len(sent) < min_len else [])
            next_token, next_prob = self._best_candidate(prev, without=blacklist)
            sent.append(next_token)
            prob *= next_prob

            if len(sent) >= max_len:
                sent.append(self.eos)

        return (' '.join(sent[(self.n-1):-1]), -1/math.log(prob))
    
    
    def add_sentence_tokens(self, sentences, n):
        return_value = []
        sos = ' '.join(self.sos * (n-1)) if n > 1 else self.sos
        for sentence in sentences:
            ids = self.tokenizer.encode(sentence).ids
            sos_id = ids[0]
            ids = ids[1:]
            s = ' '.join([str(x) for x in ids])
            return_value.append('{} {}'.format(sos, s))
        return return_value


    def preprocess(self, sentences, n):
        sentences = self.add_sentence_tokens(sentences, n)
        tokens = ' '.join(sentences).split()
        return tokens

In [47]:
for ngram_model_file in glob.glob('ngram*.pkl'):
    print(ngram_model_file)
    name_parts = ngram_model_file[:-4].split('_')
    level = name_parts[1]
    n = int(name_parts[2])
    laplace = int(name_parts[3])
    with open(ngram_model_file, 'rb') as f:
        model = pickle.load(f)
        model.set_tokenizer(tokenizers[level])
        ppl = model.perplexity(train_dataset)
        train_results[f'ngram_{n}_{laplace}'][level] = ppl

ngram_word_3_0.pkl


100%|██████████| 40000/40000 [00:14<00:00, 2785.64it/s]


ngram_bpe_3_0.pkl


100%|██████████| 40000/40000 [00:15<00:00, 2604.49it/s]


ngram_word_3_1.pkl


100%|██████████| 40000/40000 [00:14<00:00, 2785.77it/s]


ngram_bpe_3_1.pkl


100%|██████████| 40000/40000 [00:15<00:00, 2592.05it/s]


In [48]:
for ngram_model_file in glob.glob('ngram*.pkl'):
    name_parts = ngram_model_file[:-4].split('_')
    level = name_parts[1]
    n = int(name_parts[2])
    laplace = int(name_parts[3])
    with open(ngram_model_file, 'rb') as f:
        print(ngram_model_file)
        model = pickle.load(f)
        model.set_tokenizer(tokenizers[level])
        ppl = model.perplexity(test_dataset)
        test_results[f'ngram_{n}_{laplace}'][level] = ppl

ngram_word_3_0.pkl


100%|██████████| 80000/80000 [00:25<00:00, 3163.33it/s]


ngram_bpe_3_0.pkl


100%|██████████| 80000/80000 [00:27<00:00, 2932.11it/s]


ngram_word_3_1.pkl


100%|██████████| 80000/80000 [00:25<00:00, 3154.72it/s]


ngram_bpe_3_1.pkl


100%|██████████| 80000/80000 [00:27<00:00, 2931.94it/s]


# <div class="farsi right green"> لود کردن مدل LSTM و محاسبه پرپلکسیتی آن </div>

<div class="farsi right">مدل lstm استفاده شده دارای یک لایه است و طول امبدینگ ۱۰۰ برای آن در نظر گرفته شده است. اندازه لایه مخفی نیز برابر ۱۰۰ در نظر گرفته شده است.</div>
<div class="farsi right">برای ترین راحت‌تر مدل از کلیپینگ به همراه دراپ او با احتمال ۵ درصد استفاده شده است.</div>

In [13]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

EPOCHS = 300
MAX_LENGTH = 128
BPTT = 32
CLIP = 0.25
HIDDEN_SIZE = 100
EMBEDING_SIZE = 100
BATCH_SIZE = 20
INITIAL_LR = 20

class LSTMModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, ntoken, ninp, nhid, bsz):
        super().__init__()
        dropout=0.5
        nlayers = 1
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.lstm = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers
        self.bsz = bsz

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.lstm(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, self.bsz, self.nhid), weight.new_zeros(self.nlayers, self.bsz, self.nhid))

In [14]:
def calc_lstm_ppl(dataset, model, tokenizer):
    nlls = []

    sum_len = 0
    criterion = nn.NLLLoss()
    for line in tqdm.tqdm(dataset):
        ids = tokenizer.encode(line).ids
        input_ids = torch.tensor(ids[:-1]).view(-1, 1).to('cuda')
        target_ids = torch.tensor(ids[1:]).to('cuda')
        hidden = model.init_hidden()
        trg_len = len(ids)
        sum_len += trg_len
        with torch.no_grad():
            outputs, _ = model(input_ids, hidden)

            loss = criterion(outputs, target_ids)
            neg_log_likelihood = loss.item() * trg_len

        nlls.append(neg_log_likelihood)
    return torch.exp(torch.tensor(nlls).sum() / sum_len)

In [15]:
import torch

for mode in ['word', 'bpe']:
    path = f'lstm_{mode}'
    model = torch.load(path)
    model.eval()
    model.bsz = 1
    ppl = calc_lstm_ppl(train_dataset, model, tokenizers[mode])
    train_results[f'lstm'][mode] = ppl.item()

100%|██████████| 40000/40000 [00:35<00:00, 1139.48it/s]


In [16]:
for mode in ['word', 'bpe']:
    path = f'lstm_{mode}'
    model = torch.load(path)
    model.eval()
    model.bsz = 1
    ppl = calc_lstm_ppl(test_dataset, model, tokenizers[mode])
    test_results[f'lstm'][mode] = ppl.item()

100%|██████████| 80000/80000 [01:00<00:00, 1318.04it/s]


# <div class="farsi right green"> لود کردن مدل GPT2 و محاسبه پرپلکسیتی آن </div>

<div class="farsi right">مدل gpt2 استفاده شده در این پروژه یک دیکودر سه لایه است و از سایز امبدینگ ۱۲۰ در آن استفاده شده است.</div>
<div class="farsi right">این مدل دارای ۱۲ head برای اتنشن خود است.</div>

In [17]:
from transformers import GPT2LMHeadModel
import torch
import copy

def calc_gpt2_ppl(dataset, model, tokenizer):
    nlls = []
    
    sum_len = 0

    for line in tqdm.tqdm(dataset):
        ids = tokenizer.encode(line).ids
        input_ids = torch.tensor(ids).to('cuda')
        target_ids = input_ids.clone()
        trg_len = len(ids)
        sum_len += trg_len
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / sum_len)



In [18]:
for mode in ['bpe', 'word']:
    model = GPT2LMHeadModel.from_pretrained(f'gpt2_{mode}').to('cuda')
    ppl = calc_gpt2_ppl(test_dataset, model, tokenizers[mode])
    test_results[f'gpt2'][mode] = ppl.item()

100%|██████████| 80000/80000 [02:11<00:00, 610.62it/s]


In [19]:
for mode in ['bpe', 'word']:
    model = GPT2LMHeadModel.from_pretrained(f'gpt2_{mode}').to('cuda')
    ppl = calc_gpt2_ppl(train_dataset, model, tokenizers[mode])
    train_results[f'gpt2'][mode] = ppl.item()

100%|██████████| 40000/40000 [01:07<00:00, 592.54it/s]


# <div class="farsi right green">مقایسه مدل‌ها و نتایج نهایی</div>

### <div class="farsi right blue">پرپلکسیتی داده ترین</div>

In [49]:
pd.DataFrame.from_dict(train_results, orient='index')

,word,bpe
ngram_3_0,17.233117,10.362756
ngram_3_1,637.935506,2058.613123
lstm,83.147423,169.956314
gpt2,50.772160,66.034538


### <div class="farsi right blue">پرپلکسیتی داده تست</div>

In [50]:
import pandas as pd

pd.DataFrame.from_dict(test_results, orient='index')

,word,bpe
ngram_3_0,18.209904,16.458979
ngram_3_1,413.521919,1037.899298
lstm,238.585480,530.619385
gpt2,315.204773,640.165161


<div class="farsi right blue">نتایج کلی:</div>
<div class="farsi right">۱- bpe در اکثر اوقات باعث بیشتر شدن پرپلکسیتی شده است.</div>
<div class="farsi right">۲-مدل‌های مبتنی بر شبکه عصبی به علت کوچک بودن و ترینینگ کم، ضعیفتر از مدل 3gram بدون لاپلاس عمل می‌کنند.</div>
<div class="farsi right">۳- مدل lstm و gpt2 دچار مشکل شدید overfit شده‌اند.</div>
<div class="farsi right">۴- مدل دارای لاپلاس دارای بیشترین پرپلکسیتی در هر دو حالت است.</div>